<a href="https://colab.research.google.com/github/creatorabhi/Monte-Carlo-Analysis/blob/main/Validation_Level_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import os
from google.cloud import bigquery
import google.auth
import os
import pyarrow
import pandas as pd
!pip install xlsxwriter
import xlsxwriter
from itertools import compress

     |████████████████████████████████| 148 kB 5.4 MB/s 


In [7]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'/content/drive/MyDrive/Validation automation/saras-daton-8f4c72170219.json'
client = bigquery.Client()

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
os.chdir(r"/content/drive/MyDrive/Validation automation/SC reports")

In [9]:
os.listdir()

['Overall_sales.xlsx',
 'SP Advertised product report beauty.xlsx',
 'SD Advertised product report beauty.xlsx',
 'SB Campaign report beauty.xlsx']

In [10]:
ads_dict = {}

In [11]:
file_names = os.listdir()
file_names = [x.split('.')[0] for x in file_names]

In [12]:
for file in file_names:
    ads_dict[file] = pd.read_excel(os.listdir()[os.listdir().index(f"{file}.xlsx")])

ads_dict["SB Campaign report beauty"]
    

,Month,Date,Portfolio name,Currency,Campaign Name,Impressions,Clicks,Click-Thru Rate (CTR),Cost Per Click (CPC),Spend,Total Advertising Cost of Sales (ACoS),Total Return on Advertising Spend (RoAS),14 Day Total Sales,14 Day Total Orders (#),14 Day Total Units (#),14 Day Conversion Rate,14 Day New-to-brand Orders (#),14 Day % of Orders New-to-brand,14 Day New-to-brand Sales,14 Day % of Sales New-to-brand,14 Day New-to-brand Units (#),14 Day % of Units New-to-brand,14 Day New-to-brand Order Rate
0,7,2021-07-01,Not grouped,USD,CM | All Products | SBA PHRASE HOMEPAGE Test 1,145,0,0.000000,NaN,0.00,NaN,NaN,0,0,0,NaN,0,NaN,0,NaN,0,NaN,NaN
1,7,2021-07-02,Not grouped,USD,CM | All Products | SBA PHRASE HOMEPAGE Test 1,317,1,0.003155,0.740000,0.74,NaN,0.000000,0,0,0,0.000000,0,NaN,0,NaN,0,NaN,0.000000
2,7,2021-07-03,Not grouped,USD,CM | All Products | SBA PHRASE HOMEPAGE Test 1,176,1,0.005682,0.750000,0.75,NaN,0.000000,0,0,0,0.000000,0,NaN,0,NaN,0,NaN,0.000000
3,7,2021-07-04,Not grouped,USD,CM | All Products | SBA PHRASE HOMEPAGE Test 1,189,0,0.000000,NaN,0.00,NaN,NaN,0,0,0,NaN,0,NaN,0,NaN,0,NaN,NaN
4,7,2021-07-05,Not grouped,USD,CM | All Products | SBA PHRASE HOMEPAGE Test 1,162,0,0.000000,NaN,0.00,NaN,NaN,0,0,0,NaN,0,NaN,0,NaN,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181,8,2021-08-27,Sponsored Brands - Mix Products,USD,CM | All Products (BRAND) | SBA PHRASE HOMEPAGE,335,5,0.014925,1.668000,8.34,0.072522,13.788969,115,2,2,0.400000,2,1.0,115,1.0,2,1.0,0.400000
182,8,2021-08-28,Sponsored Brands - Mix Products,USD,CM | All Products (BRAND) | SBA PHRASE HOMEPAGE,336,7,0.020833,3.171429,22.20,NaN,0.000000,0,0,0,0.000000,0,NaN,0,NaN,0,NaN,0.000000
183,8,2021-08-29,Sponsored Brands - Mix Products,USD,CM | All Products (BRAND) | SBA PHRASE HOMEPAGE,354,9,0.025424,2.740000,24.66,0.632308,1.581509,39,1,1,0.111111,1,1.0,39,1.0,1,1.0,0.111111
184,8,2021-08-30,Sponsored Brands - Mix Products,USD,CM | All Products (BRAND) | SBA PHRASE HOMEPAGE,395,7,0.017722,1.971429,13.80,NaN,0.000000,0,0,0,0.000000,0,NaN,0,NaN,0,NaN,0.000000


In [13]:
# Getting required metrics from seller central SP reports

SC_SP = ads_dict[file_names[1]].fillna(0).groupby(['Month'])[["Clicks", "Impressions", "Spend","7 Day Total Sales "]].sum().reset_index()
SC_SP.rename(columns = {'7 Day Total Sales ':'Sales'}, inplace = True)
SC_SP = SC_SP.apply(pd.to_numeric, downcast='float').rename(index={0: 'July',1:'August'})
SC_SP = SC_SP.transpose()
SC_SP.name = "Seller Central SP"
SC_SP

,July,August
Month,7.000000,8.00000
Clicks,2130.000000,2004.00000
Impressions,601563.000000,552629.00000
Spend,3071.810059,3630.97998
Sales,23130.000000,22178.00000


In [14]:
# Getting required metrics from seller central SD reports

SC_SD = ads_dict[file_names[2]].fillna(0).groupby(['Month'])[["Clicks", "Impressions", "Spend","14 Day Total Sales "]].sum().reset_index()
SC_SD.rename(columns = {'14 Day Total Sales ':'Sales'}, inplace = True)
SC_SD = SC_SD.apply(pd.to_numeric, downcast='float').rename(index={0: 'July',1:'August'})
SC_SD = SC_SD.transpose()
SC_SD.name = "Seller Central SD"
SC_SD

,July,August
Month,7.000000,8.00000
Clicks,1076.000000,913.00000
Impressions,650493.000000,480185.00000
Spend,2216.879883,1816.02002
Sales,39.000000,117.00000


In [61]:
# Getting required metrics from seller central overall sales reports
ads_dict[file_names[0]] = ads_dict[file_names[0]].loc[(ads_dict[file_names[0]]['order-status'] == 'Shipped') & (ads_dict[file_names[0]]['sales-channel'] == 'Amazon.com')]


SC_Sales = ads_dict[file_names[0]].fillna(0).groupby(['Month'])[["item-price"]].sum().reset_index()
SC_Sales.rename(columns = {"item-price":"Sales"}, inplace = True)
SC_Sales = SC_Sales[SC_Sales["Month"] > 6]
SC_Sales = SC_Sales[SC_Sales["Month"] < 9]
SC_Sales = SC_Sales.apply(pd.to_numeric, downcast='float').rename(index={6: 'July',7:'August'})
SC_Sales = SC_Sales.transpose()
SC_Sales.name = "Seller Central Overall Sales"
SC_Sales

,July,August
Month,7.00,8.000
Sales,1419746.25,1116938.875


In [15]:
# Extracting the SP metrics from Biq query warehouse
query = (
"""
SELECT extract(month from reportDate) Month,sum(clicks) Clicks,sum(impressions) Impressions,sum(cost) Spend, sum(attributedSales7d) Sales 
FROM `saras-daton.data_transformation.SponsoredProducts_ProductAdsReport_Consolidated` where lower(client_name) like '%beauty%'
and extract(month from reportDate) in (7,8)  
group by 1
ORDER BY 1

    """
)
BQ_SP = client.query(query).to_dataframe()
BQ_SP = BQ_SP.apply(pd.to_numeric, downcast='float').rename(index={0: 'July',1:'August'})
BQ_SP = BQ_SP.transpose()
BQ_SP.name = "Big Query SP"
BQ_SP

,July,August
Month,7.000000,8.00000
Clicks,1854.000000,2004.00000
Impressions,549425.000000,552418.00000
Spend,2737.040039,3630.97998
Sales,20146.000000,22377.00000


In [16]:
# Extracting the SD metrics from Biq query warehouse
query = (
"""
SELECT extract(month from reportDate) Month,sum(clicks) Clicks,sum(impressions) Impressions,sum(cost) Spend, sum(attributedSales14d) Sales 
FROM `saras-daton.data_transformation.SponsoredDisplay_ProductAdsReport_Consolidated` where lower(client_name) like '%beauty%'
and extract(month from reportDate) in (7,8)  
group by 1
ORDER BY 1
"""

)
BQ_SD = client.query(query).to_dataframe()
BQ_SD = BQ_SD.apply(pd.to_numeric, downcast='float').rename(index={0: 'July',1:'August'})
BQ_SD = BQ_SD.transpose()
BQ_SD.name = "Big Query SD"
BQ_SD

,July,August
Month,7.000000,8.00000
Clicks,1047.000000,1036.00000
Impressions,635362.000000,537170.00000
Spend,2152.300049,2051.47998
Sales,39.000000,117.00000


In [17]:
# Extracting the SB metrics from Biq query warehouse
query = (
"""
SELECT extract(month from reportDate) Month,sum(clicks) Clicks,sum(impressions) Impressions,sum(cost) Spend, sum(attributedSales14d) Sales 
FROM `saras-daton.data_transformation.SponsoredBrands_CampaignsReport_Consolidated` where lower(client_name) like '%beauty%'
and extract(month from reportDate) in (7,8)  
group by 1
ORDER BY 1

    """
)
BQ_SB = client.query(query).to_dataframe()
BQ_SB = BQ_SB.apply(pd.to_numeric, downcast='float').rename(index={0: 'July',1:'August'})
BQ_SB = BQ_SB.transpose()
BQ_SB.name = "Big Query SP"
BQ_SB

,July,August
Month,7.000000,8.000000
Clicks,243.000000,244.000000
Impressions,24336.000000,15805.000000
Spend,506.390015,540.809998
Sales,3175.000000,1409.000000


In [62]:
# Extracting the overall sales metrics from Biq query warehouse
query = (
"""
SELECT extract(month from purchase_Date) Month, sum(item_price) Sales FROM `saras-daton.data_transformation.FlatFileAllOrdersReportbyLastUpdate_Consolidated`
where lower(client_name) like '%tough%'and  order_status = "Shipped"  and sales_channel = "Amazon.com" and CAST(LEFT(CAST(purchase_date as STRING), 10) AS DATE) between "2021-07-01" and "2021-08-31"
group by 1
order by 1

    """
)
BQ_Sales = client.query(query).to_dataframe()
BQ_Sales = BQ_Sales.apply(pd.to_numeric, downcast='float').rename(index={0: 'July',1:'August'})
BQ_Sales = BQ_Sales.transpose()
BQ_Sales.name = "Big Query Overall Sales"
BQ_Sales

,July,August
Month,7.0,8.000
Sales,1414816.5,1166669.625


In [19]:
diff_SP = (SC_SP.subtract(BQ_SP).div(SC_SP)).mul(100).astype('int')
diff_SP.name = "Variance of SP metrics in %"
diff_SP

,July,August
Month,0,0
Clicks,12,0
Impressions,8,0
Spend,10,0
Sales,12,0


In [20]:
diff_SD = (SC_SD.subtract(BQ_SD).div(SC_SD)).mul(100).astype('int')
diff_SD.name = "Variance of SD metrics in %"
diff_SD

,July,August
Month,0,0
Clicks,2,-13
Impressions,2,-11
Spend,2,-12
Sales,0,0


In [63]:
diff_Sales = (SC_Sales.subtract(BQ_Sales).div(SC_Sales)).mul(100).astype('int')
diff_Sales.name = "Variance of overall Sales Metrics in %"
diff_Sales

,July,August
Month,0,0
Sales,0,-4


In [64]:
writer = pd.ExcelWriter('Validation.xlsx', engine='xlsxwriter')
workbook=writer.book
worksheet=workbook.add_worksheet('Result')
writer.sheets['Result'] = worksheet

#SP
#for SC_SP to excel
worksheet.write_string(0, 0, SC_SP.name)
SC_SP.to_excel(writer,sheet_name='Result',startrow=1 , startcol=0)
#for BQ_SP to excel
worksheet.write_string(SC_SP.shape[0] + 4, 0, BQ_SP.name)
BQ_SP.to_excel(writer,sheet_name='Result',startrow=SC_SP.shape[0] + 5, startcol=0)
#for diff_SP to excel
worksheet.write_string(BQ_SP.shape[0] + 13, 0, diff_SP.name)
diff_SP.to_excel(writer,sheet_name='Result',startrow=BQ_SP.shape[0] + 14, startcol=0)

#SD
#for SC_SD to excel
worksheet.write_string(0, 5, SC_SD.name)
SC_SD.to_excel(writer,sheet_name='Result',startrow=1 , startcol=5)
#for BQ_SD to excel
worksheet.write_string(SC_SD.shape[0] + 4, 5, BQ_SD.name)
BQ_SD.to_excel(writer,sheet_name='Result',startrow=SC_SD.shape[0] + 5, startcol=5)
#for diff_SD to excel
worksheet.write_string(BQ_SD.shape[0] + 13, 5, diff_SD.name)
diff_SD.to_excel(writer,sheet_name='Result',startrow=BQ_SD.shape[0] + 14, startcol=5)

#Overall Sales
#for SC_Sales to excel
worksheet.write_string(0, 10, SC_Sales.name)
SC_Sales.to_excel(writer,sheet_name='Result',startrow=1 , startcol=10)
#for BQ_Sales to excel
worksheet.write_string(SC_Sales.shape[0] + 4, 10, BQ_Sales.name)
BQ_Sales.to_excel(writer,sheet_name='Result',startrow=SC_Sales.shape[0] + 5, startcol=10)
#for diff_Sales to excel
worksheet.write_string(BQ_Sales.shape[0] + 13, 10, diff_Sales.name)
diff_Sales.to_excel(writer,sheet_name='Result',startrow=BQ_Sales.shape[0] + 14, startcol=10)


writer.save()